# Data preparation 

In questo script i dati vengono preparati al training andandoli a bilanciare e normalizzare. Successivamente viene fatta un'operazione di split per creare train set e test set per ogni dataset.

## Import di librerie

In [3]:
import numpy as np
import dlib
import pandas as pd 
import os
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

import dlib.cuda as cuda
dlib.DLIB_USE_CUDA = True

## Funzione di conversione

Questa funzione converte le stringhe in array di NumPy

In [4]:
def converter(instr):
    return np.fromstring(instr[1:-1],sep=' ')

## Lettura dataset

In [5]:
dataset_1 = pd.read_csv("dfCelebA.csv", converters={'dizionario':converter})
dataset_2 = pd.read_csv("dfWheDataset.csv", converters={'dizionario':converter})

## Creazione dataset 3

In questa sezione vengono concatenati i due dataset (dataset1 CelebA e dataset2 WheDataset) e poi vengono mescolati tra di loro tramite la funzione sample() in modo da creare un nuovo dataset

In [6]:
frames = [dataset_1, dataset_2]
dataset_conc = pd.concat(frames)
dataset3 = dataset_conc.sample(frac=1)

## Bilanciamento dataset

In questa sezione tutti e tre i dataset prima di essere suddivisi in train e test set vengono bilancianti andando a eguagliare il numero di vettori caratteristica che rappresentano un uomo e il numero di vettori caratteristica che rappresentano una donna. In questo modo su ogni dataset abbiamo lo stesso numero di uomini e di donne.

In [7]:
#bilanciamento dataset1 - CelebA
df1_majority = dataset_1[dataset_1.gender_str==0]
df1_minority = dataset_1[dataset_1.gender_str==1]

df1_majority_downsampled = resample(df1_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=84434,     # to match minority class
                                 random_state=123) # reproducible results

df1_downsampled = pd.concat([df1_majority_downsampled, df1_minority])
dataset1_balanced = df1_downsampled.sample(frac=1)


#bilanciamento dataset2 - WeDataset
df2_majority = dataset_2[dataset_2.gender_str==1]
df2_minority = dataset_2[dataset_2.gender_str==0]

df2_majority_downsampled = resample(df2_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=36451,     # to match minority class
                                 random_state=123) # reproducible results

df2_downsampled = pd.concat([df2_majority_downsampled, df2_minority])
dataset2_balanced = df2_downsampled.sample(frac=1)



#bilanciamento dataset3 - WeDataset + CelebA
df3_majority = dataset3[dataset3.gender_str==0]
df3_minority = dataset3[dataset3.gender_str==1]

df3_majority_downsampled = resample(df3_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=148169,     # to match minority class
                                 random_state=123) # reproducible results

df3_downsampled = pd.concat([df3_majority_downsampled, df3_minority])
dataset3_balanced = df3_downsampled.sample(frac=1)

## Normalizzazione dataset

In [8]:
#Normalizzazione dataset1 - CelebA
X1 = dataset1_balanced.dizionario.to_numpy()
X1 = np.stack(X1)
X1 = normalize(X1)
y1 = dataset1_balanced.gender_str.to_numpy()

#Normalizzazione dataset2 - WheDataset
X2 = dataset2_balanced.dizionario.to_numpy()
X2 = np.stack(X2)
X2 = normalize(X2)
y2 = dataset2_balanced.gender_str.to_numpy()

#Normalizzazione dataset3 - CelebA + WheDataset
X3 = dataset3_balanced.dizionario.to_numpy()
X3 = np.stack(X3)
X3 = normalize(X3)
y3 = dataset3_balanced.gender_str.to_numpy()

## Split dataset

I tre dataset vengono suddivisi in:
- #### 70% train set
- #### 30% test set 

In [9]:
TEST_PERCENTAGE = 0.3

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=TEST_PERCENTAGE, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=TEST_PERCENTAGE, random_state=42)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=TEST_PERCENTAGE, random_state=42)


## Salvataggio train e test set

In questa sezione viene creata una cartella per ogni dataset in cui salvare il train set e il test set. Questa operazione viene fatta per avere la possibilità di fare training sempre sugli stessi dati e valutare meglio l'accuracy.

In [10]:
#Salvataggio train e test set DATASET1 (CelebA)
os.mkdir('dataset1')
np.save('dataset1/X1_train', X1_train)
np.save('dataset1/X1_test', X1_test)
np.save('dataset1/y1_train', y1_train)
np.save('dataset1/y1_test', y1_test)


#Salvataggio train e test set DATASET2 (WheDataset)
os.mkdir('dataset2')
np.save('dataset2/X2_train', X2_train)
np.save('dataset2/X2_test', X2_test)
np.save('dataset2/y2_train', y2_train)
np.save('dataset2/y2_test', y2_test)



#Salvataggio train e test set DATASET3 (CelebA + WheDataset)
os.mkdir('dataset3')
np.save('dataset3/X3_train', X3_train)
np.save('dataset3/X3_test', X3_test)
np.save('dataset3/y3_train', y3_train)
np.save('dataset3/y3_test', y3_test)